## 5강 Passage Retrieval - Dense Embedding

### 1. Introduction to Dense Embedding

#### Passage Embedding
- 구절(Passage)을 벡터로 변환하는 것

#### Limitations of sparse embedding
- 차원의 수가 매우 크다 => compressed format으로 극복 가능
- 유사성을 고려하지 못한다

#### Dense Embedding 이란?
- Complementary to sparse representations by design
    - 더 작은 차원의 고밀도 벡터 (length = 50-1000)
    - 각 차원이 특정 term에 대응되지 않음
    - 대부분의 요소가 non-zero 값
    
#### Retrieval: Sparse vs Dense
- 1) Sparse Embedding
    - 중요한 term들이 정확히 일치해야 하는 경우 성능이 뛰어남
    - 임베딩이 구축되고 나서는 추가적인 학습이 불가능함
- 2) Dense Embedding
    - 단어의 유사성 또는 맥락을 파악해야 하는 경우 성능이 뛰어남
    - 학습을 통해 임베딩을 만들며 추가적인 학습 또한 가능
    
#### Overview of Passage Retrieval with Dense Embedding
- 1) Dense embedding을 생성한 인코더 훈련
- 2) 질문과 문서를 비교하여 관련 문서 추출

### 2. Training Dense Encoder

#### What can be Dense Encoder?
- BERT와 같은 Pre-trained language model (PLM)이 자주 사용
- 그 외 다양한 neural network 구조도 가능
- BERT as dense encoder => [CLS] token의 output 사용

#### Dense Encoder 학습 목표와 학습 데이터
- **학습 목표** : 연관된 question과 passage dense embedding 간의 거리를 좁히는 것 (또는 inner product를 높이는 것). 즉 higher similarity
- **Challenge** : 연관된 qeustion / passage를 어떻게 찾을 것인가? => 기존 MRC 데이터셋을 활용


- **Negative Sampling**
    - 연관된 question과 passage간의 dense embedding 거리를 좁히는 것 (higher similarity) => ***Positive***
    - 연관 되지 않은 question과 passage간의 embedding 거리는 멀어야 함 => ***Negative***
    - **Choosing negative examples**
        - Corpus 내에서 랜덤하게 뽑기
        - 좀 더 헷갈리는 negative 샘플들 뽑기 (ex. 높은 TF-IDF 스코어를 가지지만 답을 포함하지 않는 샘플)
        

- **Objective function**
    - Positive passage에 대한 negative log likelihood (NLL) loss 사용
    
    
- **Evaluation Metric for Dense Encoder**
    - Top-k retrieval accuracy: retrieve된 passage 중에 답을 포함하는 passage의 비율
    
    
### 3. Passage Retrieval with Dense Encoder

#### From dense encoding to retrieval
- Inference: Passage와 query를 각각 embedding한 후, query로부터 가까운 순서대로 passage의 순위를 매김

#### From retrieval to open-domain question answering
- Retriever를 통해 찾아낸 Passage를 활용, MRC (Machine Reading Comprehension) 모델로 답을 찾음

#### How to make better dense encoding
- 학습 방법 개선 (e.g. DPR)
- 인코더 모델 개선 (BERT보다 큰, 정확한 Pretrained 모델)
- 데이터 개선 (더 많은 데이터, 전처리 등)

### 6강 Scaling up with FAISS

#### Passage Retrieval and Similarity Search
- **MIPS (Maximum Inner Product Search)**
    - 주어진 질문(query) 벡터 q에 대해 Passage 벡터 v들 중 가장 질문과 관련된 벡터를 찾아야함
    - 관련성(유사도)은 내적(inner product)이 가장 큰 것
    
#### MIPS & Challenges
- 실제로 검색해야할 데이터는 훨씬 방대함
- 그러므로 모든 문서 임베딩을 확인해 검색할 수 없다.

#### Tradeoffs of similarity search
- 1) Search Speed
    - 쿼리 당 유사한 벡터를 k개 찾는데 얼마나 걸리는지?
    - 가지고 있는 벡터량이 클수록 더 오래 걸림
- 2) Memory Usage
    - 벡터를 사용할 때, 어디에서 가져올 것인지?
    - RAM에 모두 올려둘 수 있으면 빠르지만, 많은 RAM 용량을 요구함
    - 디스크에서 계속 불러와야 한다면 속도가 느려짐
- 3) Accuracy
    - Brute-force 검색 결과와 얼마나 비슷한지?
    - 속도를 증가시키려면 정확도를 희생해야하는 경우가 많음
    
#### Tradeoff of search speed and accuracy
- 속도(search time)와 재현율(recall)의 관계
- 더 정확한 검색을 하려면 더 오랜 시간이 소모됨

#### Increasing search space by bigger corpus
- 코퍼스(corpus)의 크기가 커질수록
    - 탐색 공간이 커지고 검색이 어려워짐
    - 저장해 둘 Memory space 또한 많이 요구됨
    - Sparse Embedding의 경우 이러한 문제가 훨씬 심함
    
### 2. Approximating Similarity Search

#### Compression - Scalar Quantization (SQ)
- Compression : verctor를 압축하여, 하나의 vector가 적은 용량을 차지 => 압축량에 따라 메모리는 줄어들고 정보 손실은 많아진다.
    - ex) Scalar quantization : 4-byte floating point => 1-byte (8bit) unsigned interger로 압축
    
#### Pruning - Inverted File (IVF)
- Pruning : Search space를 줄여 search 속도 개선 (dataset의 subset만 방문) => Clustering + Inverted file을 활용한 search
    - 1) Clustering : 전체 vector space를 k개의 cluster로 나눔
    - 2) Inverted file : vector의 index = inverted list structure => 각 cluster의 centroid id와 해당 cluster의 vector들이 연결되어 있는 형태
    
### Introduction to FAISS

#### What is FAISS
- Faiss is a library for efficient similarity search and clusterning of dense vectors

#### Passage Retrieval with FAISS
- 1) Train index and map vectors
- 2) Search based on FAISS index

-----

### 시도한 것

#### Retrieval 수정

- **기존 retrieval**
    - mecab.morps & 1-2 gram & seed(2021)
    - max_features = 50000
    - 성능 : 0.224 / (54/240)
    
    
- **1) 기존 retrieval에서 parameter 값 변경**
    - max_features = 250000 & 1-3 gram
    - 성능 : 0.283 / (63/240)
    - max_features = 250000
    - 성능 : **0.354 / (85/240)**
    - max_features = 200000
    - 성능 : 0.354 / (85/240)
    - max_features = 150000
    - 성능 : 0.325 / (78/240)
    - max_features = 100000
    - 성능 : 0.279 / (67/240)


- **2) 전처리 후 TF-IDF 적용**
    - mecab tokenizer로 단어만 check (2글자부터) : 162646
    - mecab.nouns(2글자부터) & 1-2 gram
    - max_features = 200000
    - 성능 : 0.325 / (78/240)
    - max_features = 150000
    - 성능 : **0.338 / (81/240)**
    - max_features = 100000 
    - 성능 : 0.325 / (78/240)
    - max_features = 50000
    - 성능 : 0.283 / (68/240)
    
    
- max_features만 높이면 전처리는 크게 중요하지 않은 느낌...
- 35%로는 TEAM-IKYO 1등 못한다 다른 방법이 필요..

### 새로 알게된 것

### TODO

- TF-IDF matrix 차원 축소
- Doc2Vec or Dense Embedding